In [ ]:
import stock_data

import pandas

# get data

In [ ]:
data = stock_data.get_all_closing_prices("../stock_csvs/")

# get features

### get returns

#### closing

In [ ]:
closing_prices =  data.ix[:, data.columns.get_level_values(1).isin({"close"})]

In [ ]:
closing_returns =  (closing_prices.shift(-1) - closing_prices) / closing_prices

#### opening

In [ ]:
opening_prices =  data.ix[:, data.columns.get_level_values(1).isin({"open"})]

In [ ]:
opening_returns =  (opening_prices.shift(-1) - opening_prices) / opening_prices

#### intraday

In [ ]:
dataframes = []
for column_pair in closing_returns.columns:
    tic = column_pair[0]
    dataframes.append(
        pandas.concat({tic: 
            pandas.concat({'intra':
                          (data[tic].close - data[tic].open)
                              / data[tic].open
                          }, axis=1)}, axis=1))

In [ ]:
intra_rets = dataframes[0].join(dataframes[1:])

#### high

In [ ]:
high =  data.ix[:, data.columns.get_level_values(1).isin({"high"})]

In [ ]:
high_returns =  (high.shift(-1) - high) / high

#### low

In [ ]:
low =  data.ix[:, data.columns.get_level_values(1).isin({"loww"})]

In [ ]:
low_returns =  (low.shift(-1) - low) / low

### additional

#### volume

In [ ]:
volume =  data.ix[:, data.columns.get_level_values(1).isin({"volume"})]

In [ ]:
volume_returns =  (volume.shift(-1) - volume) / volume

#### top correlator

In [ ]:
corrs = stock_data.get_max_corrs(closing_returns)

In [ ]:
dataframes = []
for column_pair in closing_returns.columns:
    tic = column_pair[0]
    top_corr = corrs.ix[tic, 'most correlated other stock']
    dataframes.append(
        pandas.concat({tic: 
            pandas.concat({top_corr: 
                          closing_prices[top_corr].close}, axis=1)}, axis=1))

In [ ]:
all_corrs = dataframes[0].join(dataframes[1:])

### combine data

In [ ]:
dataframes = []
for column_pair in closing_returns.columns:
    tic = column_pair[0]
    
    dataframes.append(pandas.concat({
                          tic: opening_returns[tic].join(
                              closing_returns[tic]).join(
                                  high_returns[tic]).join(
                                      low_returns[tic]).join(
                                          intra_rets[tic]).join(
                                              volume_returns[tic]).join(
                                                    all_corrs[tic])
                }, axis=1))

In [ ]:
feature_table = dataframes[0].join(dataframes[1:])

In [ ]:
feature_table.head()